# Tree of Thought Reasoning

In [1]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.agents.utils import nodes_to_message_history

## The Problem

Design a file deduplication system for a cloud storage service.

This problem benefits from Tree of Thought because:
- Multiple valid algorithms exist (hashing strategies, chunking approaches)
- Each has different trade-offs (accuracy vs speed vs storage overhead)
- Requirements vary based on use case (exact duplicates vs similar files)
- Exploring alternatives prevents committing to an approach with deal-breaker limitations

In [2]:
problem = """Design a file deduplication system for a cloud storage service.

Requirements:
- Handle millions of files (documents, images, videos)
- Detect duplicate files to save storage costs
- Fast upload processing (users shouldn't wait long)
- Minimize false positives (marking different files as duplicates)
- Handle files from 1KB to 5GB

Propose different deduplication approaches."""

## Step 1: Generate Multiple Deduplication Approaches

Instead of committing to one algorithm, generate 3 different approaches to explore.

In [3]:
agent = get_agent()

prompt_generate = f"""{problem}

Generate exactly 3 different deduplication approaches. For each approach, provide:
- A name (Approach A, B, C)
- The core algorithm/technique used
- Brief description (2-3 sentences) of how it works

Keep each description concise. Focus on the fundamental differences between approaches."""

agent_run_1, nodes_1 = await run_agent(agent, prompt_generate)

assert agent_run_1 is not None and agent_run_1.result is not None
print("Generated Approaches:")
print(agent_run_1.result.output)

Generated Approaches:
# File Deduplication System Approaches

## Approach A: Content Hash-Based Deduplication
**Core Algorithm:** Cryptographic hashing (SHA-256) of entire file content

Calculates a hash of the complete file content and stores it as a unique identifier. When a new file is uploaded, its hash is computed and checked against existing hashes in the database. If a match is found, only a reference to the existing file is stored rather than duplicating the content.

## Approach B: Chunked Deduplication with Rolling Hash
**Core Algorithm:** Rabin fingerprinting with variable-size chunking

Splits files into variable-sized chunks (4KB-64KB) using content-defined boundaries based on rolling hash windows. Each chunk is hashed separately, allowing detection of partial file duplicates and efficient handling of modified files. Only unique chunks are stored, with files reconstructed from chunk references.

## Approach C: Fast Pre-filter with Perceptual Hashing
**Core Algorithm:** Two

## Step 2: Evaluate Each Approach

Score each approach on key system requirements.
This evaluation helps identify which paths are worth exploring further.

In [4]:
message_history = nodes_to_message_history(nodes_1)

prompt_evaluate = """Evaluate each of the 3 approaches you generated.

For each approach, rate it on these criteria (1-5 scale, 5 is best):
1. Accuracy (avoiding false positives/negatives)
2. Performance (speed of duplicate detection)
3. Storage overhead (metadata storage requirements)
4. Scalability (handles millions of files)

Provide a total score and brief justification for each rating.
Then rank the approaches from best to worst."""

agent_run_2, nodes_2 = await run_agent(
    agent, prompt_evaluate, message_history=message_history
)

assert agent_run_2 is not None and agent_run_2.result is not None
print("\nEvaluation:")
print(agent_run_2.result.output)


Evaluation:
# Evaluation of Deduplication Approaches

## Approach A: Content Hash-Based Deduplication

| Criterion | Rating | Justification |
|-----------|--------|---------------|
| **Accuracy** | 5/5 | SHA-256 has virtually zero collision probability; perfect detection of exact duplicates with no false positives |
| **Performance** | 3/5 | Must hash entire file (slow for 5GB files); simple database lookup is fast, but upload processing is bottlenecked by hashing |
| **Storage Overhead** | 5/5 | Minimal - only 32 bytes per file hash plus reference count; very efficient |
| **Scalability** | 4/5 | Database lookups scale well with indexing; hash computation is parallelizable, but large files still take time |

**Total Score: 17/20**

Simple and reliable, but slower for large files since entire content must be processed before deduplication decision.

---

## Approach B: Chunked Deduplication with Rolling Hash

| Criterion | Rating | Justification |
|-----------|--------|---------------

## Step 3: Expand the Top 2 Approaches

Rather than continuing all branches, prune the lowest-scoring approach.
Expand the top 2 by developing detailed implementation designs.

In [5]:
message_history = nodes_to_message_history(nodes_2)

prompt_expand = """Take the TOP 2 approaches from your ranking.
For each of these two approaches, provide a detailed implementation design:

For each approach, specify:
1. Data structures needed (what gets stored, indexed)
2. Upload flow (step-by-step what happens when a file is uploaded)
3. Query flow (how to check if a file is a duplicate)
4. Storage requirements (rough estimate for 1 million files)

Be specific with technical details."""

agent_run_3, nodes_3 = await run_agent(
    agent, prompt_expand, message_history=message_history
)

assert agent_run_3 is not None and agent_run_3.result is not None
print("\nExpanded Implementations:")
print(agent_run_3.result.output)


Expanded Implementations:
# Detailed Implementation Design for Top 2 Approaches

## Approach A: Content Hash-Based Deduplication

### 1. Data Structures

#### Database Tables

```sql
-- Main file metadata table
CREATE TABLE files (
    file_id UUID PRIMARY KEY,
    user_id UUID NOT NULL,
    filename VARCHAR(512) NOT NULL,
    content_hash CHAR(64) NOT NULL,  -- SHA-256 hex
    file_size BIGINT NOT NULL,
    mime_type VARCHAR(100),
    uploaded_at TIMESTAMP NOT NULL,
    INDEX idx_user_files (user_id, uploaded_at),
    INDEX idx_content_hash (content_hash),
    FOREIGN KEY (content_hash) REFERENCES file_contents(content_hash)
);

-- Deduplicated content storage references
CREATE TABLE file_contents (
    content_hash CHAR(64) PRIMARY KEY,  -- SHA-256 hex
    storage_path VARCHAR(1024) NOT NULL,  -- S3 key or filesystem path
    compressed_size BIGINT NOT NULL,
    reference_count INT NOT NULL DEFAULT 1,
    first_uploaded_at TIMESTAMP NOT NULL,
    last_accessed_at TIMESTAMP,
    INDE

## Step 4: Final Evaluation and Selection

With detailed implementations for the top approaches, perform a final evaluation.
Consider edge cases, failure modes, and operational complexity.

In [ ]:
message_history = nodes_to_message_history(nodes_3)

prompt_final = """Now that you have detailed implementations for the top 2 approaches:

1. Identify edge cases or failure modes for each approach
2. Consider operational complexity (monitoring, debugging, maintenance)
3. Think about how each handles the file size range (1KB to 5GB)
4. Choose the final winner
5. Explain why this approach is superior given the implementation details

Provide a clear final recommendation with specific reasoning."""

agent_run_4, nodes_4 = await run_agent(
    agent, prompt_final, message_history=message_history
)

assert agent_run_4 is not None and agent_run_4.result is not None
print("\nFinal Selection:")
print(agent_run_4.result.output)